# Анализ квитанций в службе Распознавателя документов (Form Recognizer)

![Робот держит квитанцию](./images/receipt_analysis.jpg)

В сфере искусственного интеллекта компьютерного зрения для чтения печатных или рукописных документов обычно используется оптическое распознавание символов (OCR). Часто текст просто извлекается из документов в формате, который можно использовать для дальнейшей обработки или анализа.

При более продвинутом сценарии OCR информация извлекается из таких бланков, как заказы на поставку или счета-фактуры, с семантическим пониманием того, что представляют собой поля в документе. Служба **Распознавателя документов** специально разработана для решения такого рода задач ИИ.

## Просмотреть квитанцию

В данном примере для анализа квитанций используется встроенная модель Распознавателя документов.

Нажмите кнопку **Выполнить код в ячейке** (&#9655;) (слева от ячейки) внизу, чтобы запустить ее и посмотреть пример квитанции, которую вы будете использовать для анализа с помощью Распознавателя документов.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Создайте ресурс Распознавателя документов

Давайте начнем с создания ресурса Распознавателя документов в вашей подписке Azure.

1. В другой вкладке браузера откройте портал Azure по адресу: https://portal.azure.com, войдя в систему под учетной записью Microsoft.
2. Выберите **+ Создать ресурс**, после чего найдите *Распознаватель документов*.
3. В списке служб выберите **Распознаватель документов**.
4. В колонке **Распознаватель документов** выберите **Создать**.
5. В колонке **Создать** введите следующие данные и нажмите **Создать**.
   - **Имя**: Уникальное имя для вашей службы
   - **Подписка**: Ваша подписка Azure
   - **Регион**: Любой доступный регион
   - **Ценовая категория**: классы F0
   - **Группа ресурсов**: Существующая группа ресурсов, которую вы использовали ранее
   - **Подтверждаю, что приведенное ниже уведомление прочитано и понято**: Выбрано.
6. Дождитесь завершения создания службы.
7. Просмотрите вновь созданную службу Распознавателя документов на портале Azure и на странице **Ключи и конечная точка** скопируйте значения **Ключ1** и **Конечная точка** и вставьте их в кодовую ячейку ниже, заменив **YOUR_FORM_KEY** и **YOUR_FORM_ENDPOINT**.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Проанализировать квитанцию

Теперь вы готовы использовать Распознаватель документов для анализа квитанции.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Обратите внимание, что Распознаватель документов способен интерпретировать данные в документе, правильно идентифицируя адрес и номер телефона продавца, дату и время сделки, а также строки, промежуточный итог, сумму налога и общую сумму.

## Дополнительные сведения

Дополнительные сведения о службе Распознаватель документов см. в [документации по Распознавателю документов ](https://docs.microsoft.com/ru-ru/azure/cognitive-services/form-recognizer/index)